In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from math import log, sqrt, exp
from scipy import stats
from math import log, sqrt, exp
from scipy import stats
from scipy.stats import norm
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from statistics import NormalDist

import warnings
warnings.filterwarnings("ignore")

In [2]:
class BSM:
#class constructor; input: S,vol,r,q
    def __init__(self,S0,vol,r,q):
        """class constructor; input: S0,vol,r,q"""
        self.S, self.vol, self.r, self.q = S0, vol, r, q

# alternative constructor; take data from dictionary
    @classmethod
    def from_dict( cls_, d ):
        S=d['S'] if 'S' in d else 100
        vol=d['vol'] if 'vol' in d else 0.3
        r=d['r'] if 'r' in d else 0
        q=d['q'] if 'q' in d else 0
        return cls_(S,vol,r,q)
    
    def discount(self, T):
        return np.exp(-(self.r) * T)
    
    def Binary(self, K, T):
        return (1 - self.Pr(K, T)) * 1 * self.discount(T) 
    
    def Bond(self, K, T):
        return 1 * self.discount(T) 
    
    def Call(self,K,T):
        """Calculate the price of Call Option. Inputs: Strike, Time to maturity."""
        d2=(np.log(self.S / K) + (self.r-self.q - self.vol**2 / 2) * T) / (self.vol * np.sqrt(T));
        d1=(np.log(self.S/K) + (self.r -self.q+ self.vol**2 / 2) * T)/(self.vol * np.sqrt(T))
        return np.exp(-self.q * T) *self.S * norm.cdf(d1) - K * np.exp(-self.r * T) * norm.cdf(d2)

    def Put(self,K,T):
        """Calculate the price of Put Option. Inputs: Strike, Time to maturity."""
        d2=(np.log(self.S / K) + (self.r-self.q - self.vol**2 / 2) * T) / (self.vol * np.sqrt(T));
        d1=(np.log(self.S/K) + (self.r -self.q+ self.vol**2 / 2) * T)/(self.vol * np.sqrt(T))
        return -np.exp(-self.q * T) *self.S * norm.cdf(-d1) + K * np.exp(-self.r * T) * norm.cdf(-d2)

    
    def DeltaCall(self,K,T):
        """Calculate the Delta of a Call Option. Inputs: Strike, Time to maturity."""
        d1=(np.log(self.S/K) + (self.r -self.q+ self.vol**2 / 2) * T)/(self.vol * np.sqrt(T))
        return np.exp(-self.q * T) * norm.cdf(d1)

    def DeltaPut(self,K,T):
        """Calculate the Delta of a Put Option. Inputs: Strike, Time to maturity."""
        d1=(np.log(self.S/K) + (self.r -self.q+ self.vol**2 / 2) * T)/(self.vol * np.sqrt(T))
        return np.exp(-self.q * T) * (norm.cdf(d1)-1)
    

    def Gamma(self,K,T):
        """Calculate the Gamma of an Option. Inputs: Strike, Time to maturity."""
        d1=(np.log(self.S/K) + (self.r -self.q+ self.vol**2 / 2) * T)/(self.vol * np.sqrt(T))
        return norm.pdf(d1)/(self.S*self.vol*np.sqrt(T))


    def Pr(self,K,T):
        """Calculate the probability of S(T)<K. """
        d2=(np.log(self.S / K) + (self.r-self.q - self.vol**2 / 2) * T) / (self.vol * np.sqrt(T));
        return 1-norm.cdf(d2)


    def Vega(self,K,T):
        """Calculate the vega of Call option. """
        d1=(np.log(self.S/K) + (self.r -self.q+ self.vol**2 / 2) * T)/(self.vol * np.sqrt(T))
        vega = np.exp(-self.q * T)*self.S*np.sqrt(T)*norm.pdf(d1);
        return vega

In [3]:
# 1
S_0 = 100
u = 1.1
d = 0.9
K = 90
r = 0
T = 1/12

S_0_u = S_0*u
if S_0_u < K:
    V_u = K - S_0_u
else:
    V_u = 0

print('1)')
print('S_0_u: %.2f' % S_0_u)
print('V_u: %.2f' % V_u)
print('')

S_0_d = S_0*d
if S_0_d < K:
    V_d = K - S_0_d
else:
    V_d = 0
print('S_0_d: %.2f' % S_0_d)
print('V_d: %.2f' % V_d)
print('')

print('p = delta*S_0 + L, where')
delta = (V_u - V_d) / (S_0*(u-d))
L = (V_d*u - V_u*d) / ((1 + r*T)*(u-d))
p = delta*S_0 + L
print('delta = %.2f' % delta)
print('L = %.2f' % L)
print('Since p = %.2f' % p)
print('')
print('2) Arbitrage')
print('If the market price of an option is $5, but the real price is $0, we should sell this option.')
print('In that case we get $5.')

1)
S_0_u: 110.00
V_u: 0.00

S_0_d: 90.00
V_d: 0.00

p = delta*S_0 + L, where
delta = 0.00
L = 0.00
Since p = 0.00

2) Arbitrage
If the market price of an option is $5, but the real price is $0, we should sell this option.
In that case we get $5.


In [4]:
# 2
S_0, sigma, r, N, T, K = 100, 0.3, 0.02, 1000, 0.5, 90
d_2 = (1/(sigma*np.sqrt(T)))*(np.log(S_0/K) + (r - sigma**2/2)*T)
a = stats.norm.cdf(-d_2)
digital_put_price = exp(-r*T)*a*N

print('Digital put price: %.4f' % digital_put_price)

Digital put price: 327.4929


In [5]:
## 3
## 1 вопрос. Как реплицировать структурную ноту при помощи депозитов и опционов?
## 2 вопрос. Вега - ?, S_0 = 4500 USD, r = 2%, q = 2%, сигма = 20%, T = 1 год.

## Ответ на 1 вопрос:
## 1 шаг. Кладём на депозит такую часть из 100 000 под процентную ставку r, чтобы 
## этот депозит обеспечил возврат 100%, то есть 100 000.
## 2 шаг. На оставшуюся сумму денег покупаем в продукт опционную часть,
## которая может принести потенциальный доход.
## Входные данные: Вся инвестиция 100% = Депозит + Купленный опцион.
## Итог: Депозит (= 100%) + Условный доход (>= 0%).

I_sum = 100000
r = 0.02
q = 0.02
T = 1
S_0 = 4500
sigma = 0.2


print('1 шаг.')
depozit = I_sum / (1 + r)
print('На депозит кладём: %.4f' % depozit)
print('')
print('2 шаг.')
print(f'На оставшуюся сумму денег {I_sum - depozit} покупаем call S_0 = {S_0}.')
A = BSM(4500, 0.2, 0.02, 0.02)
call_price = A.Call(4500, 1)
a_o = (I_sum - depozit) / call_price
print('The price of one call option: %.4f' % call_price)
print('Since we can buy %.4f' % a_o,'options, i.e.')
print('{0:.4f}'.format(a_o*S_0), 'i.e. {0:.4f}'.format(a_o*S_0/I_sum*100),'%.' )
print('Справедливый коэффициент участия: {0:.2f}'.format(a_o*S_0/I_sum*100), '%')

1 шаг.
На депозит кладём: 98039.2157

2 шаг.
На оставшуюся сумму денег 1960.7843137254968 покупаем call S_0 = 4500.
The price of one call option: 351.3527
Since we can buy 5.5807 options, i.e.
25113.0230 i.e. 25.1130 %.
Справедливый коэффициент участия: 25.11 %


In [6]:
# 4
# 1 часть. Европейский колл-опцион (покупка):
# K_1 = 4600 USD, T_1 = 0.5 года, S_0 = 4500 USD, sigma = 20%, r = 2%, q = 0
# Найти Вега
A = BSM(4500, 0.2, 0.02, 0)
Vega_1 = A.Vega(4600, 0.5)/100
print('Vega_1: {0:.3f}'.format(Vega_1))

# 2 часть. Европейский пут:
# K_2 = 4300 USD, T_2 = 1 год.
Vega_2 = A.Vega(4300, 1)/100
print('Vega_2: {0:.3f}'.format(Vega_2))
print('Vega hedging: {0:.4f}'.format( -Vega_1/Vega_2))
print('That means we should sell 0.7746 of a put option.')

Vega_1: 12.693
Vega_2: 16.386
Vega hedging: -0.7746
That means we should sell 0.7746 of a put option.


In [7]:
# 5
# Трёхшаговая модель, 1 шаг = 1/12 года, put - ?, delta в каждом узле - ?

In [8]:
S_0 = 100
r = 0.03 # простые проценты без капитализации
u = 1.1
d = 0.9
K = 100 # american put
T = 1/12
print('1)')
print('{0:.2f}'.format(S_0))
a = S_0*u
b = S_0*d
print('{0:.2f}'.format(a), '{0:.2f}'.format(b))
c = a*u
m = a*d
e = b*d
print('{0:.2f}'.format(c), '{0:.2f}'.format(m), '{0:.2f}'.format(e))
f = c*u
g = c*d
k = m*d
l = e*d
print('{0:.2f}'.format(f), '{0:.2f}'.format(g), '{0:.2f}'.format(k), '{0:.2f}'.format(l))
print('2)')
print('Calculate risk-neutral probability, using the formula q = (1+rT-d) / (u-d)')
q = (1+r*T-d) / (u-d)
print('q: {0:.2f}'.format(q))
print('1-q: {0:.2f}'.format(1-q))
print('')
print('The prob tree')
print(1)
print('{0:.2f}'.format(q), '{0:.2f}'.format(1-q))
print('{0:.2f}'.format(q**2), '{0:.2f}'.format(q*(1-q)), '{0:.2f}'.format((1-q)**2))
print('{0:.2f}'.format(q**3), '{0:.2f}'.format(q**2*(1-q)), '{0:.2f}'.format(q*(1-q)**2), '{0:.2f}'.format((1-q)**3))
print('')

print('3)')
print('Calculate the option price for European Call in each knot, using the formula (qV_u + (1-q)V_d) / (1 + rT)')
# put K = 100, if S_t > K => 0, elsewhere K - S_t
print('{0:.2f}'.format(0), '{0:.2f}'.format(0), '{0:.2f}'.format(K-k), '{0:.2f}'.format(K-l))
c_1 = (q*0 + (1-q)*0) / (1 + r*T)
m_1 = (q*0 + (1-q)*(K-k)) / (1 + r*T)
e_1 = (q*(K-k) + (1-q)*(K-l)) / (1 + r*T)
print('{0:.2f}'.format(c_1), '{0:.2f}'.format(m_1), '{0:.2f}'.format(e_1))
a_1 = (q*c_1 + (1-q)*m_1) / (1 + r*T)
b_1 = (q*m_1 + (1-q)*e_1) / (1 + r*T)
print('{0:.2f}'.format(a_1), '{0:.2f}'.format(b_1))
S_0_1 = (q*a_1 + (1-q)*b_1) / (1 + r*T)
print('{0:.2f}'.format(S_0_1))
print('')

print('4)')
print('Find delta in each knot, using formula (V_u - V_d) / (S_0(u-d))')
delta_1 = 0
delta_2 = 0
delta_3 = 1
delta_4 = 1
print('{0:.2f}'.format(delta_1), '{0:.2f}'.format(delta_2), '{0:.2f}'.format(delta_3), '{0:.2f}'.format(delta_4))
delta_5 = (0 - 0) / (f-g)
delta_6 = (0 - (K-k)) /(g-k)
delta_7 = ((K-k) - (K-l)) / (k-l)
print('{0:.2f}'.format(delta_5), '{0:.2f}'.format(delta_6), '{0:.2f}'.format(delta_7))
delta_8 = (c_1 - m_1) / (c-m)
delta_9 = (m_1 - e_1) / (m-e)
print('{0:.2f}'.format(delta_8), '{0:.2f}'.format(delta_9))
delta_10 = (a_1 - b_1) / (a-b)
print('{0:.2f}'.format(delta_10))
print('')

print('5)')
print('Выгодно исполнить put optin на 2 шаге, так как это принесёт профит 100-81=19, вместо 18.75, тогда')
print('for American option')
print('{0:.2f}'.format(0), '{0:.2f}'.format(0), '{0:.2f}'.format(K-k), '{0:.2f}'.format(K-l))
c_1_2 = (q*0 + (1-q)*0) / (1 + r*T)
m_1_2 = (q*0 + (1-q)*(K-k)) / (1 + r*T)
e_1_2 = 100 - 81
print('{0:.2f}'.format(c_1_2), '{0:.2f}'.format(m_1_2), '{0:.2f}'.format(e_1_2))
a_1_2 = (q*c_1_2 + (1-q)*m_1_2) / (1 + r*T)
b_1_2 = (q*m_1_2 + (1-q)*e_1_2) / (1 + r*T)
print('{0:.2f}'.format(a_1_2), '{0:.2f}'.format(b_1_2))
S_0_1_2 = (q*a_1_2 + (1-q)*b_1_2) / (1 + r*T)
print('{0:.2f}'.format(S_0_1_2))
print('Именно поэтому американский опцион будет стоить дороже.')
print('')

print('6)')
print('Если процентная ставка станет нулевой, то разницы между американсим и европейским опционами нет.')

1)
100.00
110.00 90.00
121.00 99.00 81.00
133.10 108.90 89.10 72.90
2)
Calculate risk-neutral probability, using the formula q = (1+rT-d) / (u-d)
q: 0.51
1-q: 0.49

The prob tree
1
0.51 0.49
0.26 0.25 0.24
0.13 0.13 0.12 0.12

3)
Calculate the option price for European Call in each knot, using the formula (qV_u + (1-q)V_d) / (1 + rT)
0.00 0.00 10.90 27.10
0.00 5.30 18.75
2.58 11.83
7.07

4)
Find delta in each knot, using formula (V_u - V_d) / (S_0(u-d))
0.00 0.00 1.00 1.00
0.00 -0.55 -1.00
-0.24 -0.75
-0.46

5)
Выгодно исполнить put optin на 2 шаге, так как это принесёт профит 100-81=19, вместо 18.75, тогда
for American option
0.00 0.00 10.90 27.10
0.00 5.30 19.00
2.58 11.95
7.13
Именно поэтому американский опцион будет стоить дороже.

6)
Если процентная ставка станет нулевой, то разницы между американсим и европейским опционами нет.


In [9]:
# 6
S_0 = 100
r = 0.02
sigma = 0.2
K = 110
B = 130
T = 0.5
print('1)')
A = BSM(100, 0.2, 0.02, 0)
call_1 = A.Call(110, 0.5)
print('Call_1: {0:.2f}'.format(call_1))
print('')
print('2)')
call_2 = A.Call(130, 0.5)
print('Call_2: {0:.2f}'.format(call_2))
print('')
print('3)')

d_2 = (np.log(S_0 / B) + (r - sigma**2 / 2) * T) / (sigma * np.sqrt(T))
digital_put = exp(-r*T)*norm.cdf(d_2)*(B-K)

print('Digital call: {0:.2f}'.format(digital_put))
print('')
print('4)')
print('EKO = Long Call (call_1) - Short Call (call_2) - Digital Put')
EKO = call_1 - call_2 - digital_put
print('EKO: {0:.2f}'.format(EKO))
print('')
print('5)')
print('EKO дешевле, чем обычный ванильный европейский колл на {0:.2f}'.format(call_1-EKO))

1)
Call_1: 2.47

2)
Call_2: 0.24

3)
Digital call: 0.63

4)
EKO = Long Call (call_1) - Short Call (call_2) - Digital Put
EKO: 1.61

5)
EKO дешевле, чем обычный ванильный европейский колл на 0.87


In [10]:
# 7

A = BSM(100, 0.3, 0.02, 0)

Delta_1 = A.DeltaCall(110, 0.25)
c_Call_1 = A.Call(110, 0.25)
c_Call_1_K_2 = A.Call(100, 1)



Delta_2 = A.DeltaCall(100, 1)
Gamma_1 = A.Gamma(110, 0.25)
Gamma_2 = A.Gamma(100, 1)
Gamma_ratio = Gamma_1 / Gamma_2
B = BSM(102, 0.3, 0.02, 0)
Delta_1_102 = B.DeltaCall(110, 0.25)
c_Call_1_102 = B.Call(110, 0.25)
c_Call_1_102_K_2 = B.Call(100, 1)

print('1)')
print('Sold Delta Call_1: {0:.4f}'.format(Delta_1))
print('Since we should buy 0.299 of базовый актив.')
print('')
print('Если цена акции выросла до 102 USD, то')
print('- Рост цены акции приводит к увеличению коэффициента дельта.')
print('- Чтобы сбалансировать необходимо дополнительно купить')
print('  (102 - 100)*(0.299 of базовый актив) = {0:.4f}'.format((102-100)*0.299), '(1)')
print('')
print('- При продаже Call_1 получаем премию.')
print('  c_Call_1: {0:.4f}'.format(c_Call_1))
print('  то есть: +{0:.4f}'.format(c_Call_1), '(2)')
print('')
print('- Call_1_102 стоит')
print('  c_Call_1_102: {0:.4f}'.format(c_Call_1_102))
print('  то есть: {0:.4f}'.format(-c_Call_1_102), '(3)')
print('')
print('В итоге мы теряем (1) + (2) + (3) = {0:.4f}'.format((102-100)*0.299 + c_Call_1 - c_Call_1_102))

print('')
print('2)')
print('Delta Call_2: {0:.4f}'.format(Delta_2))
print('Gamma Call_1: {0:.4f}'.format(Gamma_1))
print('Gamma Call_2: {0:.4f}'.format(Gamma_2))
print('The ratio of the Gammas: {0:.4f}'.format(Gamma_ratio))
a = -Delta_1 + (Gamma_ratio)*Delta_2
print('The residual Delta: {0:.4f}'.format(a))
print('The answer is for Delta and Gamma hedging we need to buy {0:.4f}'.format(Gamma_ratio), 
      'units of Call_2 and sell', '{0:.4f}'.format(a), 'of stock.')
print('')
print('Если цена акции выросла до 102 USD, то')
print('- Чтобы сбалансировать необходимо продать')
print('  (102 - 100)*(0.7447 of базовый актив) = {0:.4f}'.format((102-100)*a))
print('  то есть: {0:.4f}'.format(-(102-100)*a), '(1)')
print('')
print('- При продаже Call_1 со страйком K_1 получаем премию.')
print('  c_Call_1: {0:.4f}'.format(c_Call_1))
print('  то есть: +{0:.4f}'.format(c_Call_1), '(2)')
print('')
print('- Call_1_102 со страйком K_1 стоит')
c_Call_1_102 = B.Call(110, 0.25)
print('  c_Call_1_102: {0:.4f}'.format(c_Call_1_102))
print('  то есть: {0:.4f}'.format(-c_Call_1_102), '(3)')
print('')
print('- При покупке Call_1 со страйком K_2 платим премию.')
print('  c_Call_1_K_2: {0:.4f}'.format(c_Call_1_K_2*Gamma_ratio))
print('  то есть: {0:.4f}'.format(-c_Call_1_K_2*Gamma_ratio), '(4)')
print('')
print('- Call_1_102_K_2 со страйком K_2 стоит')
c_Call_1_102 = B.Call(110, 0.25)
print('  c_Call_1_102_K_2: {0:.4f}'.format(c_Call_1_102_K_2*Gamma_ratio))
print('  то есть: +{0:.4f}'.format(c_Call_1_102_K_2*Gamma_ratio), '(5)')
res = -(102-100)*a + c_Call_1 - c_Call_1_102 - c_Call_1_K_2*Gamma_ratio + c_Call_1_102_K_2*Gamma_ratio
print('')
print('В итоге мы теряем (1) + (2) + (3) + (4) + (5) = {0:.4f}'.format(res))

1)
Sold Delta Call_1: 0.2991
Since we should buy 0.299 of базовый актив.

Если цена акции выросла до 102 USD, то
- Рост цены акции приводит к увеличению коэффициента дельта.
- Чтобы сбалансировать необходимо дополнительно купить
  (102 - 100)*(0.299 of базовый актив) = 0.5980 (1)

- При продаже Call_1 получаем премию.
  c_Call_1: 2.6341
  то есть: +2.6341 (2)

- Call_1_102 стоит
  c_Call_1_102: 3.2792
  то есть: -3.2792 (3)

В итоге мы теряем (1) + (2) + (3) = -0.0472

2)
Delta Call_2: 0.5858
Gamma Call_1: 0.0231
Gamma Call_2: 0.0130
The ratio of the Gammas: 1.7820
The residual Delta: 0.7447
The answer is for Delta and Gamma hedging we need to buy 1.7820 units of Call_2 and sell 0.7447 of stock.

Если цена акции выросла до 102 USD, то
- Чтобы сбалансировать необходимо продать
  (102 - 100)*(0.7447 of базовый актив) = 1.4895
  то есть: -1.4895 (1)

- При продаже Call_1 со страйком K_1 получаем премию.
  c_Call_1: 2.6341
  то есть: +2.6341 (2)

- Call_1_102 со страйком K_1 стоит
  c_Call

### 2 часть.

In [ ]:
Структура:
    1. К задаче написан класс Parametric_Volatility, который вычисляет все внутренние переходы.
    2. В решении следуюшие разделы:
        Premium (current PV)
        1. DN + 0.1%
        2. RR25 + 0.1%
        3. FLY25 + 0.1%
        4. RR10 + 0.1%
        5. FLY10 + 0.1%
        The results
    
    3. Чтобы изменить волатильность, после класса есть соответсвующий массив.  

In [11]:
class Parametric_Volatility:
    
    
    
    def __init__(self, Vol): 
        self.Vol = Vol  
        self.S_0, self.T, self.r, self.q, self.F = 1.1130, 0.25, 0.01, -0.005, 1.1172
        self.Payout, self.Low_K_1, self.High_K_2, self.Vol_bump, self.Epsilon = 10**6, 1.08, 1.16, 0.001, 0.0001
        self.S = [1.1130]
        self.Strike_1_2 = [self.Low_K_1, self.Low_K_1 + self.Epsilon, self.High_K_2, self.High_K_2 + self.Epsilon]
        self.df1 = self.df_1() # атрибуты объекта

        
    def Call_Delta(self):
        Call_Delta = [0.9, 0.75, 0.5, 0.25, 0.1]
        return Call_Delta
    
    def Strike_1_2(self):
        print()
        Strike_1_2 = [self.Low_K_1, self.Low_K_1 + self.Epsilon, self.High_K_2, self.High_K_2 + self.Epsilon]
        return Strike_1_2
        
        
    def df(self):
        return pd.DataFrame({'Spot (S)' : self.S,
                             'Maturity (T)' : self.T,
                             'USD rate(r)' : self.r,
                             'EUR rate (q)' : self.q,
                             'Forward (F)': self.F})
    
    def Vol_sg_vla(self):
       
        return [(self.Vol[0] + self.Vol[4] - self.Vol[2]/2), 
                (self.Vol[0] + self.Vol[3] - self.Vol[1]/2), 
                (self.Vol[0]), 
                (self.Vol[0] + self.Vol[3] + self.Vol[1]/2), 
                (self.Vol[0] + self.Vol[4] + self.Vol[2]/2)]
    
    def Strike_s(self):
        
        dict_new = zip(self.Vol_sg_vla(), self.Call_Delta())
        Strike_s = []
        for i, j in dict_new:
            rev = NormalDist().inv_cdf(j * np.exp(self.q * self.T))
            Strike_s.append(self.S_0 / ( np.exp(rev*i*sqrt(self.T) - (self.r - self.q + i**2/2)*self.T)))
            
        return Strike_s
    
    def Delta_c(self):
        
        dict_new_2 = zip(self.Vol_sg_vla(), self.Strike_s())
        Delta_c = []
        for i, j in dict_new_2:
            Delta_c.append(np.exp(-self.q*self.T)*norm.cdf(1/(i*np.sqrt(self.T))*np.log(self.S_0/j) + (self.r - self.q + i**2/2)*self.T)*100)
            
        return Delta_c
    
    
    
    def Moneyness(self):
        Moneyness = []
        for i in self.Strike_s():
            Moneyness.append(np.log(i/self.F))
            
        return Moneyness
    
    def Moneyness_2(self):
        Moneyness_2 = []
        for i in self.Moneyness():
            Moneyness_2.append(i**2/10**4)
            
        return Moneyness_2
    
    def df_1(self):

        return pd.DataFrame({'Strategy' : ['DN', 'RR25', 'RR10', 'FLY25', 'FlY10'],
                             'Volatility' : Vol,
                             'Strike_sg_vla' : ['10P', '25P', 'DN', '25C', '10C'],
                             'Volatility_sg_vla' : self.Vol_sg_vla(),
                             'Call Delta' : self.Call_Delta(),
                             'Strike (solve)' : self.Strike_s(),
                             'Delta (check)' : self.Delta_c(),
                             'Moneyness' : self.Moneyness(),
                             'Moneyness^2' : self.Moneyness_2()})
    
    def A_B_C(self):
        
        X = self.df1[['Moneyness', 'Moneyness^2']]
        y = self.Vol_sg_vla()
        X = sm.add_constant(X)

        model = OLS(y, X)
        res = model.fit()
        
        A = res.params[0]
        B = res.params[1]
        C = res.params[2]
        
        return A, B, C

    def Volatility_1_2(self):
        Volatility_1_2 = []
        for i in self.Strike_1_2:            
            Volatility_1_2.append(self.A_B_C()[0] + self.A_B_C()[1]*np.log(i/self.F) + self.A_B_C()[2]*(np.log(i/self.F)**2)/10**4)
            
        return Volatility_1_2
    
    def d_1_2(self):
        
        dict_new_3 = zip(self.Strike_1_2, self.Volatility_1_2())
        d_1_2 = []
        d_2_2 = []
        for i, j in dict_new_3:
            d_1_2.append((np.log(self.S_0/i) + (self.r-self.q+j**2/2)*self.T)/(j*np.sqrt(self.T)))
            d_2_2.append((np.log(self.S_0/i) + (self.r-self.q-j**2/2)*self.T)/(j*np.sqrt(self.T)))
            
        return d_1_2, d_2_2
    
    def Vanilla_call_1(self):
        dict_new_4 = zip(self.Strike_1_2, self.d_1_2()[0], self.d_1_2()[1])
        Vanilla_call_1 = []
        for i, j, k in dict_new_4:
            Vanilla_call_1.append((self.S_0*np.exp(-self.q*self.T)*norm.cdf(j) - i*np.exp(-self.r*self.T)*norm.cdf(k))*self.Payout)
            
        return Vanilla_call_1
    
    def df_2(self):
        return pd.DataFrame({'' : ['K1', 'K1 + eps', 'K2', 'K2 + eps'],
                             'Strike' : self.Strike_1_2, 
                             'Volatility ' : self.Volatility_1_2(),
                             'd_1' : self.d_1_2()[0],
                             'd_2' : self.d_1_2()[1],
                             'Vanilla call' : self.Vanilla_call_1()})
    
    def df_3(self):
        digital_K_1 = (self.df_2().iloc[0,5] - self.df_2().iloc[1,5])/self.Epsilon
        digital_K_2 = (self.df_2().iloc[2,5] - self.df_2().iloc[3,5])/self.Epsilon
        EDNT = digital_K_1 - digital_K_2
        return pd.DataFrame({'Digital K1' : [digital_K_1],
                             'Digital K2' : [digital_K_2],
                             'EDNT' : [EDNT]})

In [12]:
Vol_bump = 0.001
Vol = [0.0735, -0.0105, -0.0205, 0.0025, 0.0095]

Vol_1 = [Vol[0] + Vol_bump, Vol[1], Vol[2], Vol[3], Vol[4]]

Vol_2 = [Vol[0], Vol[1] + Vol_bump, Vol[2], Vol[3], Vol[4]]

Vol_3 = [Vol[0], Vol[1], Vol[2], Vol[3] + Vol_bump, Vol[4]]

Vol_4 = [Vol[0], Vol[1], Vol[2] + Vol_bump, Vol[3], Vol[4]]

Vol_5 = [Vol[0], Vol[1], Vol[2], Vol[3], Vol[4] + Vol_bump]

### Premium (current PV)

In [13]:
Vol = [0.0735, -0.0105, -0.0205, 0.0025, 0.0095]
A = Parametric_Volatility(Vol)  

In [14]:
Vol = [0.0735, -0.0105, -0.0205, 0.0025, 0.0095]
A = Parametric_Volatility(Vol)  
display(A.df())
display(A.df_1())
display(A.df_2())
display(A.df_3())
Current_EDNT = A.df_3().iloc[0, 2]

,Spot (S),Maturity (T),USD rate(r),EUR rate (q),Forward (F)
0,1.113,0.25,0.01,-0.005,1.1172


,Strategy,Volatility,Strike_sg_vla,Volatility_sg_vla,Call Delta,Strike (solve),Delta (check),Moneyness,Moneyness^2
0,DN,0.0735,10P,0.09325,0.90,1.053841,88.135688,-0.058384,3.408738e-07
1,RR25,-0.0105,25P,0.08125,0.75,1.088013,71.434492,-0.026473,7.008079e-08
2,RR10,-0.0205,DN,0.07350,0.50,1.118001,45.377652,0.000716,5.131382e-11
3,FLY25,0.0025,25C,0.07075,0.25,1.144914,21.363421,0.024504,6.004492e-08
4,FlY10,0.0095,10C,0.07275,0.10,1.171299,8.098781,0.047287,2.236101e-07


,,Strike,Volatility,d_1,d_2,Vanilla call
0,K1,1.0800,0.082918,0.837148,0.795689,42392.410060
1,K1 + eps,1.0801,0.082885,0.835238,0.793796,42308.658845
2,K2,1.1600,0.071584,-1.032930,-1.068722,3070.643481
3,K2 + eps,1.1601,0.071588,-1.035277,-1.071071,3056.979923


,Digital K1,Digital K2,EDNT
0,837512.149086,136635.573768,700876.575317


### 1. DN + 0.1%

In [15]:
Vol_1
A = Parametric_Volatility(Vol_1)  
display(A.df_1())
display(A.df_2())
display(A.df_3())
DN_EDNT = A.df_3().iloc[0, 2]

,Strategy,Volatility,Strike_sg_vla,Volatility_sg_vla,Call Delta,Strike (solve),Delta (check),Moneyness,Moneyness^2
0,DN,0.0735,10P,0.09425,0.90,1.053194,88.148219,-0.058999,3.480829e-07
1,RR25,-0.0105,25P,0.08225,0.75,1.087670,71.464908,-0.026788,7.176029e-08
2,RR10,-0.0205,DN,0.07450,0.50,1.118022,45.422806,0.000736,5.411360e-11
3,FLY25,0.0025,25C,0.07175,0.25,1.145321,21.399708,0.024860,6.180004e-08
4,FlY10,0.0095,10C,0.07375,0.10,1.172071,8.116299,0.047947,2.298903e-07


,,Strike,Volatility,d_1,d_2,Vanilla call
0,K1,1.0800,0.083789,0.828885,0.786991,42529.167619
1,K1 + eps,1.0801,0.083756,0.826986,0.785108,42445.718134
2,K2,1.1600,0.072557,-1.018587,-1.054865,3198.528155
3,K2 + eps,1.1601,0.072561,-1.020907,-1.057187,3184.525256


,Digital K1,Digital K2,EDNT
0,834494.851827,140028.991844,694465.859983


### 2. RR25 + 0.1%

In [16]:
Vol_2
A = Parametric_Volatility(Vol_2)  
display(A.df_1())
display(A.df_2())
display(A.df_3())
RR_25_EDNT = A.df_3().iloc[0, 2]

,Strategy,Volatility,Strike_sg_vla,Volatility_sg_vla,Call Delta,Strike (solve),Delta (check),Moneyness,Moneyness^2
0,DN,0.0735,10P,0.09325,0.90,1.053841,88.135688,-0.058384,3.408738e-07
1,RR25,-0.0105,25P,0.08075,0.75,1.088184,71.418926,-0.026315,6.924800e-08
2,RR10,-0.0205,DN,0.07350,0.50,1.118001,45.377652,0.000716,5.131382e-11
3,FLY25,0.0025,25C,0.07125,0.25,1.145118,21.381710,0.024682,6.091916e-08
4,FlY10,0.0095,10C,0.07275,0.10,1.171299,8.098781,0.047287,2.236101e-07


,,Strike,Volatility,d_1,d_2,Vanilla call
0,K1,1.0800,0.082802,0.838267,0.796866,42374.188948
1,K1 + eps,1.0801,0.082768,0.836353,0.794969,42290.437678
2,K2,1.1600,0.071749,-1.030464,-1.066339,3092.256384
3,K2 + eps,1.1601,0.071754,-1.032798,-1.068675,3078.606766


,Digital K1,Digital K2,EDNT
0,837512.698307,136496.182292,701016.516016


### 3. FLY25 + 0.1%

In [17]:
Vol_3
A = Parametric_Volatility(Vol_3)  
display(A.df_1())
display(A.df_2())
display(A.df_3())
FLY25_EDNT = A.df_3().iloc[0, 2]

,Strategy,Volatility,Strike_sg_vla,Volatility_sg_vla,Call Delta,Strike (solve),Delta (check),Moneyness,Moneyness^2
0,DN,0.0735,10P,0.09325,0.90,1.053841,88.135688,-0.058384,3.408738e-07
1,RR25,-0.0105,25P,0.08225,0.75,1.087670,71.464908,-0.026788,7.176029e-08
2,RR10,-0.0205,DN,0.07350,0.50,1.118001,45.377652,0.000716,5.131382e-11
3,FLY25,0.0025,25C,0.07175,0.25,1.145321,21.399708,0.024860,6.180004e-08
4,FlY10,0.0095,10C,0.07275,0.10,1.171299,8.098781,0.047287,2.236101e-07


,,Strike,Volatility,d_1,d_2,Vanilla call
0,K1,1.0800,0.083395,0.832599,0.790902,42467.218821
1,K1 + eps,1.0801,0.083363,0.830688,0.789006,42383.756746
2,K2,1.1600,0.071902,-1.028194,-1.064145,3112.284001
3,K2 + eps,1.1601,0.071905,-1.030552,-1.066504,3098.338111


,Digital K1,Digital K2,EDNT
0,834620.751724,139458.89791,695161.853814


### 4. RR10 + 0.1%

In [18]:
Vol_4
A = Parametric_Volatility(Vol_4)  
display(A.df_1())
display(A.df_2())
display(A.df_3())
RR10_EDNT = A.df_3().iloc[0, 2]

,Strategy,Volatility,Strike_sg_vla,Volatility_sg_vla,Call Delta,Strike (solve),Delta (check),Moneyness,Moneyness^2
0,DN,0.0735,10P,0.09275,0.90,1.054164,88.129283,-0.058077,3.372965e-07
1,RR25,-0.0105,25P,0.08125,0.75,1.088013,71.434492,-0.026473,7.008079e-08
2,RR10,-0.0205,DN,0.07350,0.50,1.118001,45.377652,0.000716,5.131382e-11
3,FLY25,0.0025,25C,0.07075,0.25,1.144914,21.363421,0.024504,6.004492e-08
4,FlY10,0.0095,10C,0.07325,0.10,1.171685,8.107608,0.047617,2.267390e-07


,,Strike,Volatility,d_1,d_2,Vanilla call
0,K1,1.0800,0.082730,0.838956,0.797591,42363.004155
1,K1 + eps,1.0801,0.082697,0.837038,0.795690,42279.269394
2,K2,1.1600,0.071868,-1.028705,-1.064638,3107.770286
3,K2 + eps,1.1601,0.071873,-1.031031,-1.066968,3094.116570


,Digital K1,Digital K2,EDNT
0,837347.610319,136537.159037,700810.451282


### 5. FLY10 + 0.1%

In [19]:
Vol_5
A = Parametric_Volatility(Vol_5)  
display(A.df_1())
display(A.df_2())
display(A.df_3())
FLY10_EDNT = A.df_3().iloc[0, 2]

,Strategy,Volatility,Strike_sg_vla,Volatility_sg_vla,Call Delta,Strike (solve),Delta (check),Moneyness,Moneyness^2
0,DN,0.0735,10P,0.09425,0.90,1.053194,88.148219,-0.058999,3.480829e-07
1,RR25,-0.0105,25P,0.08125,0.75,1.088013,71.434492,-0.026473,7.008079e-08
2,RR10,-0.0205,DN,0.07350,0.50,1.118001,45.377652,0.000716,5.131382e-11
3,FLY25,0.0025,25C,0.07075,0.25,1.144914,21.363421,0.024504,6.004492e-08
4,FlY10,0.0095,10C,0.07375,0.10,1.172071,8.116299,0.047947,2.298903e-07


,,Strike,Volatility,d_1,d_2,Vanilla call
0,K1,1.0800,0.083001,0.836360,0.794860,42405.293486
1,K1 + eps,1.0801,0.082965,0.834466,0.792983,42321.318111
2,K2,1.1600,0.072104,-1.025218,-1.061270,3138.746940
3,K2 + eps,1.1601,0.072110,-1.027512,-1.063567,3125.249059


,Digital K1,Digital K2,EDNT
0,839753.755312,134978.810684,704774.944628


### The results

In [20]:
df_4 = pd.DataFrame({'Scenario' : ['EDNT PV', 'Vega'] ,
                     'Current' : [Current_EDNT, ''],
                     'DN' : [DN_EDNT, (DN_EDNT - Current_EDNT)*0.01/Vol_bump],
                     'RR25' : [RR_25_EDNT, (RR_25_EDNT - Current_EDNT)*0.01/Vol_bump],
                     'FLY' : [FLY25_EDNT, (FLY25_EDNT - Current_EDNT)*0.01/Vol_bump],
                     'RR10' : [RR10_EDNT, (RR10_EDNT - Current_EDNT)*0.01/Vol_bump],
                     'FLY10' : [FLY10_EDNT, (FLY10_EDNT - Current_EDNT)*0.01/Vol_bump]
                    })

df_4.transpose()

,0,1
Scenario,EDNT PV,Vega
Current,700876.575317,
DN,694465.859983,-64107.153341
RR25,701016.516016,1399.406985
FLY,695161.853814,-57147.215027
RR10,700810.451282,-661.240353
FLY10,704774.944628,38983.693107
